## Pulling From AWS

In [1]:
import os

spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

--2023-03-16 22:34:27--  https://jdbc.postgresql.org/download/postgresql-42.2.17.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004734 (981K) [application/java-archive]
Saving to: ‘postgresql-42.2.17.jar’

postgresql-42.2.17. 100%[===================>] 981.19K  6.32MB/s    in 0.2s    

2023-03-16 22:34:28 (6.32 MB/s) - ‘postgresql-42.2.17.jar’ saved [1004734/1004734]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://casey-elizabeth-haley-kaleb-bucket.s3.us-east-2.amazonaws.com/Total+Emissions+Per+Country+.csv"
spark.sparkContext.addFile(url)
total_emissions_uncleaned_df = spark.read.csv(SparkFiles.get("Total+Emissions+Per+Country+.csv"), sep=",", header=True, inferSchema=True)

## Data Cleaning

In [5]:
total_emissions_uncleaned_df.show()

+-----------+--------------------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|       Area|                Item|       Emission_Type|      Unit|Year_2000|Year_2001|Year_2002|Year_2003|Year_2004|Year_2005|Year_2006|Year_2007|Year_2008|Year_2009|Year_2010|Year_2011|Year_2012|Year_2013|Year_2014|Year_2015|Year_2016|Year_2017|Year_2018|Year_2019|Year_2020|
+-----------+--------------------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Afghanistan|       Crop Residues|Direct emissions ...|kilotonnes|     0.52|   0.5267|     0.82|   0.9988|   0.8225|   1.1821|   1.0277|   1.2426|   0.8869|    1.392|   

In [6]:
total_emissions_dropna_df = total_emissions_uncleaned_df.dropna()
total_emissions_dropna_df.show()

+-----------+--------------------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+
|       Area|                Item|       Emission_Type|      Unit|Year_2000|Year_2001|Year_2002|Year_2003|Year_2004|Year_2005|Year_2006|Year_2007|Year_2008|Year_2009|Year_2010|Year_2011|Year_2012|Year_2013|Year_2014| Year_2015| Year_2016| Year_2017| Year_2018| Year_2019| Year_2020|
+-----------+--------------------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+
|Afghanistan|       Crop Residues|Direct emissions ...|kilotonnes|     0.52|   0.5267|     0.82|   0.9988|   0.8225|   1.1821|   1.0277|   1.2426|   0.

In [7]:
total_emissions_cleaned_df = total_emissions_dropna_df.replace("kilotonnes", "kilotons")
total_emissions_cleaned_df.show()

+-----------+--------------------+--------------------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+
|       Area|                Item|       Emission_Type|    Unit|Year_2000|Year_2001|Year_2002|Year_2003|Year_2004|Year_2005|Year_2006|Year_2007|Year_2008|Year_2009|Year_2010|Year_2011|Year_2012|Year_2013|Year_2014| Year_2015| Year_2016| Year_2017| Year_2018| Year_2019| Year_2020|
+-----------+--------------------+--------------------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+
|Afghanistan|       Crop Residues|Direct emissions ...|kilotons|     0.52|   0.5267|     0.82|   0.9988|   0.8225|   1.1821|   1.0277|   1.2426|   0.8869|   

## Summary DataFrame

In [8]:
import pandas as pd

In [9]:
emissions_pandas_df = total_emissions_cleaned_df.toPandas()

In [10]:
emissions_pandas_df

,Area,Item,Emission_Type,Unit,Year_2000,Year_2001,Year_2002,Year_2003,Year_2004,Year_2005,...,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020
0,Afghanistan,Crop Residues,Direct emissions (N2O),kilotons,0.5200,0.5267,0.8200,0.9988,0.8225,1.1821,...,1.0321,1.3726,1.4018,1.4584,1.2424,1.1940,1.0617,0.8988,1.2176,1.3170
1,Afghanistan,Crop Residues,Indirect emissions (N2O),kilotons,0.1170,0.1185,0.1845,0.2247,0.1851,0.2660,...,0.2322,0.3088,0.3154,0.3281,0.2795,0.2687,0.2389,0.2022,0.2740,0.2963
2,Afghanistan,Crop Residues,Emissions (N2O),kilotons,0.6370,0.6452,1.0045,1.2235,1.0075,1.4481,...,1.2643,1.6815,1.7173,1.7865,1.5220,1.4627,1.3005,1.1011,1.4916,1.6133
3,Afghanistan,Crop Residues,Emissions (CO2eq) from N2O (AR5),kilotons,168.8070,170.9884,266.1975,324.2195,266.9995,383.7498,...,335.0379,445.5958,455.0727,473.4174,403.3181,387.6130,344.6447,291.7838,395.2689,427.5284
4,Afghanistan,Crop Residues,Emissions (CO2eq) (AR5),kilotons,168.8070,170.9884,266.1975,324.2195,266.9995,383.7498,...,335.0379,445.5958,455.0727,473.4174,403.3181,387.6130,344.6447,291.7838,395.2689,427.5284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38426,Sweden,Drained organic soils (N2O),Emissions (N2O),kilotons,5.1092,5.1209,5.1355,5.1857,5.2963,5.4384,...,6.0675,6.1048,6.1144,6.1765,6.1881,6.2629,6.3629,6.7063,6.9126,6.9338
38427,Sweden,Drained organic soils (N2O),Emissions (N2O),kilotons,3.0553,3.0473,3.0351,3.0228,3.0142,3.0049,...,2.9036,2.8808,2.8557,2.8395,2.8303,2.8156,2.7989,2.7878,2.7743,2.7589
38428,Sweden,Drained organic soils (N2O),Emissions (CO2eq) from N2O (AR5),kilotons,1353.9379,1357.0467,1360.8961,1374.2134,1403.5229,1441.1851,...,1607.8907,1617.7663,1620.3241,1636.7712,1639.8453,1659.6792,1686.1590,1777.1796,1831.8376,1837.4575
38429,Sweden,Drained organic soils (N2O),Emissions (CO2eq) from N2O (AR5),kilotons,809.6447,807.5436,804.2915,801.0535,798.7694,796.3083,...,769.4588,763.4116,756.7598,752.4800,750.0283,746.1282,741.7152,738.7774,735.1846,731.1023


In [17]:
emissions_summary_df_pandas = emissions_pandas_df.describe()
emissions_summary_df_pandas.insert(0, 'Index', emissions_summary_df_pandas.index)
emissions_summary_df_pandas

,Index,Year_2000,Year_2001,Year_2002,Year_2003,Year_2004,Year_2005,Year_2006,Year_2007,Year_2008,...,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020
count,count,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,...,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04,3.843100e+04
mean,mean,8.062607e+03,8.008313e+03,8.359142e+03,8.604091e+03,9.173096e+03,9.405770e+03,9.905405e+03,9.963879e+03,1.020617e+04,...,1.081792e+04,1.102873e+04,1.111563e+04,1.139590e+04,1.143808e+04,1.139673e+04,1.158284e+04,1.180528e+04,1.201421e+04,1.173653e+04
std,std,9.433529e+04,9.591183e+04,1.022490e+05,1.127878e+05,1.252867e+05,1.361241e+05,1.478449e+05,1.555475e+05,1.642628e+05,...,1.969400e+05,2.020808e+05,2.058135e+05,2.084398e+05,2.073283e+05,2.043913e+05,2.084539e+05,2.134644e+05,2.188329e+05,2.203599e+05
min,min,-7.410051e+05,-7.971831e+05,-7.971831e+05,-7.971831e+05,-7.971831e+05,-7.971831e+05,-7.971831e+05,-7.971831e+05,-8.156754e+05,...,-8.648649e+05,-8.781114e+05,-8.509092e+05,-8.451650e+05,-8.390716e+05,-8.239425e+05,-8.099242e+05,-7.982900e+05,-7.849816e+05,-7.729378e+05
25%,25%,2.650000e-02,1.900000e-02,2.015000e-02,2.350000e-02,2.200000e-02,2.440000e-02,2.510000e-02,2.885000e-02,2.660000e-02,...,2.810000e-02,3.100000e-02,2.945000e-02,3.100000e-02,3.400000e-02,3.365000e-02,3.605000e-02,3.380000e-02,3.715000e-02,3.060000e-02
50%,50%,6.916000e+00,6.178700e+00,6.434500e+00,6.905300e+00,6.861000e+00,7.178900e+00,7.342500e+00,7.746800e+00,7.675100e+00,...,8.245500e+00,8.406400e+00,8.387400e+00,8.467200e+00,8.816000e+00,9.029100e+00,9.155800e+00,8.946500e+00,9.391600e+00,9.087500e+00
75%,75%,3.779087e+02,3.616760e+02,3.752926e+02,3.950836e+02,3.989841e+02,4.205209e+02,4.215623e+02,4.387105e+02,4.451113e+02,...,4.629759e+02,4.645124e+02,4.759622e+02,4.775167e+02,4.913850e+02,5.044345e+02,5.106735e+02,5.111268e+02,5.199407e+02,5.058742e+02
max,max,5.383231e+06,5.484582e+06,5.925897e+06,6.731423e+06,7.577858e+06,8.375211e+06,9.128191e+06,9.683548e+06,1.021723e+07,...,1.250354e+07,1.279649e+07,1.309592e+07,1.325110e+07,1.313829e+07,1.298856e+07,1.323641e+07,1.352664e+07,1.389463e+07,1.409145e+07


In [18]:
emissions_summary_df = spark.createDataFrame(emissions_summary_df_pandas)

In [19]:
emissions_summary_df.show()

+-----+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|Index|         Year_2000|        Year_2001|         Year_2002|         Year_2003|         Year_2004|         Year_2005|         Year_2006|         Year_2007|        Year_2008|           Year_2009|           Year_2010|         Year_2011|         Year_2012|           Year_2013|         Year_2014|         Year_2015|         Year_2016|        Year_2017|         Year_2018|         Year_2019|         Year_2020|
+-----+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----

In [15]:
from getpass import getpass
password = getpass('Enter database password')

mode = "append"
jdbc_url="jdbc:postgresql://group-4-database.c0mbmuajokyj.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [16]:
total_emissions_cleaned_df.write.jdbc(url=jdbc_url, table='total_emissions', mode=mode, properties=config)

In [20]:
emissions_summary_df.write.jdbc(url=jdbc_url, table='emissions_summary', mode=mode, properties=config)